<a href="https://colab.research.google.com/github/Jamess200/BirdnetProject/blob/main/BirdNETAudio_To_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Mount Google Drive
# Mount Google Drive to access files stored in it
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#@title Change Directory
# Change the working directory to a specific path in your Google Drive
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [4]:
#@title Set Up Repository Path & Clone Repository
import os
# Define path in Google Drive where you want to clone the repository
repo_path = '/content/drive/MyDrive/'

# Check if directory already exists, create it if not
if not os.path.exists(repo_path):
    os.makedirs(repo_path)

# Change working directory to defined path
os.chdir(repo_path)

# Clone the repository if doesnt exist, otherwise pull latest changes
if not os.path.exists(os.path.join(repo_path, 'BirdnetProject')):
    !git clone https://github.com/Jamess200/BirdnetProject
else:
    os.chdir('BirdnetProject')
    !git pull

# Verify cloned repository
!ls

remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 12 (delta 4), reused 5 (delta 2), pack-reused 0
Unpacking objects: 100% (12/12), 7.40 KiB | 7.00 KiB/s, done.
From https://github.com/Jamess200/BirdnetProject
   ca26a34..4ab1b10  main       -> origin/main
Updating ca26a34..4ab1b10
Fast-forward
 Files/Lost_Data_Prediction.ipynb | 376 +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
 1 file changed, 376 insertions(+)
 create mode 100644 Files/Lost_Data_Prediction.ipynb
Files  README.md


In [5]:
#@title Install Required Python Packages
# Install required packages
!pip3 install birdnetlib
!pip3 install tflite-runtime
!pip3 install resampy
!pip3 install ffmpeg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 24.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=96256b5671f85f98de6cdb433d3bd6cc17e69e2e8316a4baa5027a3fe92e3b17
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg


In [6]:
#@title Import Necessary Libraries
# Import necessary libraries
import resampy
import birdnetlib
import tflite_runtime
import ffmpeg
import pandas as pd
from birdnetlib import Recording
from birdnetlib.analyzer import Analyzer
from datetime import datetime

In [7]:
#@title Set Up Directory and File Paths
# Define the directory where the data is stored
dataDir = '/content/drive/MyDrive/XC/'

# List all files in the directory
file_names = os.listdir(dataDir)

# Print the file names to verify the data files
print(file_names)

['XC871476 - Soundscape.wav']


In [8]:
#@title Initialise BirdNET Analyzer
# Initialise the BirdNET-Analyzer model
analyzer = Analyzer()
# Define the coordinates and date for the recordings
testamp = [52.911,-2.4441] # near Market Drayton year=2024, month=4, day=20

Labels loaded.
load model True
Model loaded.
Labels loaded.
load_species_list_model
Meta model loaded.


In [9]:
#@title Analyse Audio Files and Store Results
# Create a dictionary to store the results
results = {}
# Loop through all file names
for file_name in file_names:
    try:
      # Include the correct path to the data files
      file_path = os.path.join(dataDir, file_name)
      recording = Recording(
          analyzer,
          file_path,  # path to the recording file
          lat=testamp[0], # latitude of the recording location
          lon=testamp[1], # longitude of the recording location
          date=datetime(year=2024, month=4, day=20), # use date or week_48
          min_conf=0.25,
      )
      # Analyse the recording and store the result in the dictionary
      recording.analyze()
      results[file_name] = recording.detections
    except Exception as e:
      print(f"Error processing {file_name}: {e}")

read_audio_data
read_audio_data: complete, read  35 chunks.
analyze_recording XC871476 - Soundscape.wav
recording has lon/lat
set_predicted_species_list_from_position
return_predicted_species_list
15
145 species loaded.


In [10]:
#@title Convert Results to DataFrame and Save as CSV
# Flatten the results dictionary
flattened_data = []
for filename, records in results.items():
    for record in records:
        record['filename'] = filename
        flattened_data.append(record)

# Convert the flattened data to a DataFrame
df = pd.DataFrame(flattened_data)

# Save the DataFrame to a CSV file
csv_file_path = '/content/drive/MyDrive/BirdnetProject/Files/data/CSV_data/XC/test.csv'
df.to_csv(csv_file_path, index=False)

# Confirm that the results have been saved successfully
print(f"Results have been successfully converted to CSV and saved to {csv_file_path}")

Results have been successfully converted to CSV and saved to /content/drive/MyDrive/BirdnetProject/Files/data/CSV_data/XC/test.csv


In [11]:
#@title Load and Verify CSV Data
# Load the saved CSV file into a DataFrame to verify the contents
csv_file_path = '/content/drive/MyDrive/BirdnetProject/Files/data/CSV_data/XC/test.csv'
df = pd.read_csv(csv_file_path)

# Print the DataFrame to verify the data
print(df.head)

<bound method NDFrame.head of            common_name          scientific_name  start_time  end_time  \
0    Eurasian Blackcap       Sylvia atricapilla         3.0       6.0   
1    Eurasian Blackcap       Sylvia atricapilla         6.0       9.0   
2    Eurasian Blackcap       Sylvia atricapilla        12.0      15.0   
3   Eurasian Blackbird            Turdus merula        18.0      21.0   
4    Eurasian Blackcap       Sylvia atricapilla        21.0      24.0   
5    Eurasian Blackcap       Sylvia atricapilla        24.0      27.0   
6    Eurasian Blackcap       Sylvia atricapilla        27.0      30.0   
7    Eurasian Blackcap       Sylvia atricapilla        30.0      33.0   
8                 Rook        Corvus frugilegus        33.0      36.0   
9    Eurasian Blackcap       Sylvia atricapilla        33.0      36.0   
10   Eurasian Blackcap       Sylvia atricapilla        39.0      42.0   
11   Eurasian Blackcap       Sylvia atricapilla        42.0      45.0   
12   Eurasian Blackca